In [1]:
import importlib
import sys
from pathlib import Path
import os
import json

In [2]:
project_root = Path.cwd().parents[1]
src_path = project_root / "src"
sys.path.insert(0, str(project_root))

In [3]:
from src.config import llm_config
importlib.reload(llm_config)
from src.config.llm_config import get_llm

## Test data loader

In [ ]:
from src.agents.nodes import data_loader_agent
importlib.reload(data_loader_agent)
from src.agents.nodes.data_loader_agent import DataLoader

In [20]:
print("\n" + "="*60)
print("Testing Agent 1: Data Loader")
print("="*60)


Testing Agent 1: Data Loader


In [21]:
llm = get_llm()
agent = DataLoader(llm)

In [22]:
csv_path = os.path.join(project_root, 'inputs', 'dummy_options.csv')
csv_path = os.path.abspath(csv_path)
csv_path

'/Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv'

In [23]:
print(f"\nTest CSV path: {csv_path}")
print(f"File exists: {os.path.exists(csv_path)}")


Test CSV path: /Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv
File exists: True


In [ ]:
initial_state = {
    "messages": [],
    "csv_file_path": csv_path,
    "csv_data": None,
    "data_loader_agent_status": None,
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": None,
    "workflow_status": "started",
    "errors": []
}

# Run agent
print("\n" + "-"*60)
print("Running Data Loader Agent...")
print("-"*60)


------------------------------------------------------------
Running Data Loader Agent...
------------------------------------------------------------


In [25]:
result = agent(initial_state)

In [27]:
print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('agent1_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")


Agent execution completed!

Agent Status: completed
Current Agent: agent1
Workflow Status: in_progress


## Test calculator

In [9]:
from src.agents.nodes import data_loader_agent
importlib.reload(data_loader_agent)
from src.agents.nodes.calculator_agent import Calculator

In [10]:
print("\n" + "="*60)
print("Testing Agent 2: Calculator")
print("="*60)


Testing Agent 2: Calculator


In [11]:
llm = get_llm()
agent = Calculator(llm)

In [ ]:
mock_csv_data = {
        "option_type": {0: "call", 1: "put"},
        "S": {0: 100.0, 1: 105.0},
        "K": {0: 100.0, 1: 100.0},
        "T": {0: 1.0, 1: 0.5},
        "r": {0: 0.05, 1: 0.05},
        "sigma": {0: 0.2, 1: 0.25}
    }
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    "calculation_results": None,
    "greeks_data": None,
    "calculator_agent_status": None,
    "test_results": None,
    "test_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": None,
    "workflow_status": "started",
    "errors": []
}

# Run agent
print("\n" + "-"*60)
print("Running Calculator Agent...")
print("-"*60)
print(f"\nInput CSV Data:")
print(json.dumps(mock_csv_data, indent=2))


------------------------------------------------------------
Running Calculator Agent...
------------------------------------------------------------

Input CSV Data:
{
  "option_type": {
    "0": "call",
    "1": "put"
  },
  "S": {
    "0": 100.0,
    "1": 105.0
  },
  "K": {
    "0": 100.0,
    "1": 100.0
  },
  "T": {
    "0": 1.0,
    "1": 0.5
  },
  "r": {
    "0": 0.05,
    "1": 0.05
  },
  "sigma": {
    "0": 0.2,
    "1": 0.25
  }
}


In [13]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('calculator_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")

Testing: Running sensitivity test for call with base S=100.0, K=100.0, T=1.0, r=0.05, sigma=0.2

Agent execution completed!

Agent Status: completed
Current Agent: calculator
Workflow Status: in_progress


## Test testor

In [4]:
from src.agents.nodes import tester_agent
importlib.reload(tester_agent)
from src.agents.nodes.tester_agent import Tester

In [5]:
print("\n" + "="*60)
print("Testing Agent 3: Tester (Test Agent)")
print("="*60)


Testing Agent 3: Tester (Test Agent)


In [16]:
llm = get_llm()
agent = Tester(llm)

In [7]:
mock_csv_data = {
    "option_type": {0: "call", 1: "put"},
    "S": {0: 100.0, 1: 105.0},
    "K": {0: 100.0, 1: 100.0},
    "T": {0: 1.0, 1: 0.5},
    "r": {0: 0.05, 1: 0.05},
    "sigma": {0: 0.2, 1: 0.25}
}

mock_calculation_results = """
| option_type | S     | K     | price   | delta | gamma | vega  |
|-------------|-------|-------|---------|-------|-------|-------|
| call        | 100.0 | 100.0 | 10.45   | 0.637 | 0.019 | 39.70 |
| put         | 105.0 | 100.0 | 3.15    | -0.245| 0.021 | 28.43 |
"""


In [8]:
initial_state = {
    "messages": [],
    "csv_file_path": "/inputs/dummy_options.csv",
    "csv_data": mock_csv_data,
    "data_loader_agent_status": "completed",
    "calculation_results": mock_calculation_results,
    "greeks_data": None,
    "sensitivity_data": None,
    "calculator_agent_status": "completed",
    "test_results": None,
    "tester_agent_status": None,
    "summary_text": None,
    "summarty_writer_agnet_status": None,
    "charts": None,
    "chart_descriptions": None,
    "chart_generator_agent_status": None,
    "final_report_path": None,
    "final_report_html": None,
    "report_assembler_agent_status": None,
    "current_agent": "agent2",
    "workflow_status": "in_progress",
    "errors": []
}
print("\n" + "-"*60)
print("Running Test Agent...")
print("-"*60)
print("\nThis agent will execute actual pytest test functions:")
print("  - test_greeks_call_basic")
print("  - test_greeks_put_call_parity")
print("  - test_sensitivity_length_and_fields")


------------------------------------------------------------
Running Test Agent...
------------------------------------------------------------

This agent will execute actual pytest test functions:
  - test_greeks_call_basic
  - test_greeks_put_call_parity
  - test_sensitivity_length_and_fields


In [9]:
result = agent(initial_state)

print("\nAgent execution completed!")
print(f"\nAgent Status: {result.get('tester_agent_status')}")
print(f"Current Agent: {result.get('current_agent')}")
print(f"Workflow Status: {result.get('workflow_status')}")


Agent execution completed!

Agent Status: error
Current Agent: tester
Workflow Status: None


In [11]:
state = initial_state
csv_data = state.get("csv_data")

In [12]:
csv_data

{'option_type': {0: 'call', 1: 'put'},
 'S': {0: 100.0, 1: 105.0},
 'K': {0: 100.0, 1: 100.0},
 'T': {0: 1.0, 1: 0.5},
 'r': {0: 0.05, 1: 0.05},
 'sigma': {0: 0.2, 1: 0.25}}

In [14]:
from langchain_core.messages import HumanMessage

task_message = HumanMessage(content=f"""
Run validation tests on the option calculations.

Execute the following tests from src/core:
1. run_greeks_validation_test - runs actual pytest functions:
   - test_greeks_call_basic: validates basic Greeks calculation (delta in [0,1], price > 0)
   - test_greeks_put_call_parity: validates put-call parity relationship

2. run_sensitivity_analysis_test - runs actual pytest function:
   - test_sensitivity_length_and_fields: validates sensitivity analysis returns 11 data points with all required fields

These tools wrap the actual test functions from test_greeks.py and test_sensitivity.py.

Report the test results including pass/fail status for each test.
""")

In [18]:
result = agent.agent.invoke({
    **state,
    "messages": [task_message]
})

In [19]:
result.get("messages", [])

[HumanMessage(content='\nRun validation tests on the option calculations.\n\nExecute the following tests from src/core:\n1. run_greeks_validation_test - runs actual pytest functions:\n   - test_greeks_call_basic: validates basic Greeks calculation (delta in [0,1], price > 0)\n   - test_greeks_put_call_parity: validates put-call parity relationship\n\n2. run_sensitivity_analysis_test - runs actual pytest function:\n   - test_sensitivity_length_and_fields: validates sensitivity analysis returns 11 data points with all required fields\n\nThese tools wrap the actual test functions from test_greeks.py and test_sensitivity.py.\n\nReport the test results including pass/fail status for each test.\n', additional_kwargs={}, response_metadata={}, id='e261a339-19d3-4c54-b478-3fca2ff8a48d'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-10-28T19:47:26.227307Z', 'done': True, 'done_reason': 'stop', 'total_duration': 953052958, 'load_durati

In [21]:
from src.core.test_greeks import test_greeks_call_basic, test_greeks_put_call_parity
from src.core.bsm_calculator import greeks_calculator